In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.patches as patches
import csv as csv

In [2]:
#!pip install import_ipynb

In [2]:
import Metrica_IO as mio
import Metrica_Viz as mviz

In [3]:
def to_single_playing_direction(home,away,events):
    '''
    Flip coordinates in second half so that each team always shoots in the same direction through the match.
    '''
    for team in [home,away,events]:
        if (team.Period == 2).any():
          second_half_idx = team[team['Period'] == 2].index[0]

          # Select columns ending with 'x' or 'y' (case-insensitive)
          columns = [c for c in team.columns if c[-1].lower() in ['x', 'y']]

          # Flip the coordinates for the second half
          team.loc[second_half_idx:, columns] *= -1
    return home,away,events

In [9]:
DATADIR = '../sample-data/data/'
game_id = 1

events = mio.read_event_data(DATADIR,game_id)

tracking_home = mio.tracking_data(DATADIR,game_id,'Home')
tracking_away = mio.tracking_data(DATADIR,game_id,'Away')

tracking_home = mio.to_metric_coordinates(tracking_home)
tracking_away = mio.to_metric_coordinates(tracking_away)

events = mio.to_metric_coordinates(events)

# 전반, 후반 방향 일치 (후반전 선수 방향 filp)
tracking_home, tracking_away, events = to_single_playing_direction(tracking_home, tracking_away, events)

Reading team: home
Reading team: away


In [10]:
tracking_home.to_csv('../sample-data/data/tracking_home.csv', index=False)
tracking_away.to_csv('../sample-data/data/tracking_away.csv', index=False)
events.to_csv('../sample-data/data/events.csv', index=False)

In [ ]:
PLOTDIR = '/content/drive/MyDrive/SportsData/'
mviz.save_match_clip(tracking_home.iloc[73600:73600+500],tracking_away.iloc[73600:73600+500],PLOTDIR,fname='home_goal_2',include_player_velocities=False)

Generating movie...done


In [8]:
DATADIR2 = './interpolation'
home_list = []
away_list = []

for i in range(1, 12):
    df = pd.read_csv(f'{DATADIR2}/home_{i}.csv')
    home_list.append(df)

for i in range(1, 12):
    df = pd.read_csv(f'{DATADIR2}/away_{i}.csv')
    away_list.append(df)

home_list[0].head()

,frame,x,y,time
0,0,0.912181,0.5,0.00
1,1,0.912181,0.5,0.04
2,2,0.912181,0.5,0.08
3,3,0.913598,0.5,0.12
4,4,0.915014,0.5,0.16


In [17]:
for h in home_list:
    h['period'] = 1
for a in away_list:
    a['period'] = 1

In [14]:
"""
df_6['period'] = (df_6['time'] > 2696.48).astype(int) + 1
df_7['period'] = (df_7['time'] > 2696.48).astype(int) + 1
df_4['period'] = (df_4['time'] > 2696.48).astype(int) + 1
"""
home_list

[     frame         x         y   time  period
 0        0  0.912181  0.500000   0.00       1
 1        1  0.912181  0.500000   0.04       1
 2        2  0.912181  0.500000   0.08       1
 3        3  0.913598  0.500000   0.12       1
 4        4  0.915014  0.500000   0.16       1
 ..     ...       ...       ...    ...     ...
 613    613  0.911827  0.548458  24.52       1
 614    614  0.912181  0.548458  24.56       1
 615    615  0.912181  0.550661  24.60       1
 616    616  0.912181  0.548458  24.64       1
 617    617  0.912181  0.546256  24.68       1
 
 [618 rows x 5 columns],
      frame         x         y   time  period
 0        0  0.609065  0.493392   0.00       1
 1        1  0.609065  0.495595   0.04       1
 2        2  0.609537  0.495595   0.08       1
 3        3  0.610009  0.495595   0.12       1
 4        4  0.610482  0.495595   0.16       1
 ..     ...       ...       ...    ...     ...
 621    621  0.592776  0.654185  24.84       1
 622    622  0.593484  0.654185  

In [18]:
for i in range(1,12):
    home_list[i-1] = home_list[i-1].rename(columns={'x': f'Home_{i}_x', 'y': f'Home_{i}_y'})
    away_list[i-1] = away_list[i-1].rename(columns={'x': f'Away_{i}_x', 'y': f'Away_{i}_y'})

home_merged = home_list[0][['frame', 'period', 'time', 'Home_1_x', 'Home_1_y']]  # 데이터 병합 (frame, period, time)
away_merged = away_list[0][['frame', 'period', 'time', 'Away_1_x', 'Away_1_y']]

for i in range(2, 12):
    home_merged = home_merged.merge(
        home_list[i-1][['frame', f'Home_{i}_x', f'Home_{i}_y']],
        on='frame'
    )
    away_merged = away_merged.merge(
        away_list[i-1][['frame', f'Away_{i}_x', f'Away_{i}_y']],
        on='frame'
    )

# Frame을 인덱스로 설정
home_merged = home_merged.set_index('frame')
home_merged.index.name = 'Frame'
away_merged = away_merged.set_index('frame')
away_merged.index.name = 'Frame'
print(home_merged)
print(away_merged)

       period   time  Home_1_x  Home_1_y  Home_2_x  Home_2_y  Home_3_x  \
Frame                                                                    
0           1   0.00  0.912181  0.500000  0.609065  0.493392  0.508499   
1           1   0.04  0.912181  0.500000  0.609065  0.495595  0.508499   
2           1   0.08  0.912181  0.500000  0.609537  0.495595  0.508499   
3           1   0.12  0.913598  0.500000  0.610009  0.495595  0.508499   
4           1   0.16  0.915014  0.500000  0.610482  0.495595  0.508499   
...       ...    ...       ...       ...       ...       ...       ...   
606         1  24.24  0.910765  0.548458  0.600850  0.671806  0.431404   
607         1  24.28  0.910765  0.548458  0.600567  0.671806  0.430797   
608         1  24.32  0.910765  0.548458  0.599150  0.671806  0.430190   
609         1  24.36  0.910765  0.548458  0.600567  0.669604  0.429583   
610         1  24.40  0.910765  0.548458  0.599858  0.670705  0.428976   

       Home_3_y  Home_4_x  Home_4_y  

In [19]:
home_merged['ball_x'] = 0
home_merged['ball_y'] = 0
away_merged['ball_x'] = 0
away_merged['ball_y'] = 0

In [20]:
home_merged = home_merged.rename(columns={'period': 'Period', 'time': 'Time [s]'})
away_merged = away_merged.rename(columns={'period': 'Period', 'time': 'Time [s]'})

In [21]:
away_merged.head()

,Period,Time [s],Away_1_x,Away_1_y,Away_2_x,Away_2_y,Away_3_x,Away_3_y,Away_4_x,Away_4_y,...,Away_8_x,Away_8_y,Away_9_x,Away_9_y,Away_10_x,Away_10_y,Away_11_x,Away_11_y,ball_x,ball_y
Frame,,,,,,,,,,,,,,,,,,,,,
0,1,0.00,0.117564,0.5,0.487252,0.814978,0.371105,0.790749,0.495751,0.495595,...,0.381020,0.566079,0.385269,0.418502,0.417847,0.588106,0.488669,0.187225,0,0
1,1,0.04,0.117564,0.5,0.487252,0.814978,0.371105,0.790749,0.498584,0.500000,...,0.381020,0.563877,0.386686,0.418502,0.419263,0.588106,0.488669,0.187225,0,0
2,1,0.08,0.117564,0.5,0.487252,0.814978,0.371105,0.790749,0.498584,0.497797,...,0.381020,0.566079,0.386686,0.418502,0.419263,0.588106,0.488669,0.187225,0,0
3,1,0.12,0.117564,0.5,0.487252,0.814978,0.371105,0.790749,0.498584,0.500000,...,0.381020,0.566079,0.385269,0.420705,0.417847,0.588106,0.488669,0.187225,0,0
4,1,0.16,0.117564,0.5,0.487252,0.814978,0.371813,0.790749,0.498584,0.497797,...,0.378187,0.572687,0.385269,0.420705,0.419263,0.588106,0.488669,0.187225,0,0


In [25]:
print(home_merged.index)
print(away_merged.index)
print(home_merged.shape)
print(away_merged.shape)

Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       601, 602, 603, 604, 605, 606, 607, 608, 609, 610],
      dtype='int64', name='Frame', length=611)
Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       587, 588, 589, 590, 591, 592, 593, 594, 595, 596],
      dtype='int64', name='Frame', length=597)
(611, 26)
(597, 26)


In [27]:

home_merged = home_merged.iloc[:597]
home_merged.shape

(597, 26)

In [28]:
df_home = mio.to_metric_coordinates(home_merged)
df_away = mio.to_metric_coordinates(away_merged)


In [30]:
from scipy.signal import savgol_filter

# 예시: 각 좌표 컬럼에 필터 적용
for col in df_home.columns:
    if "_x" in col or "_y" in col:
        df_home[col] = savgol_filter(df_home[col], window_length=25, polyorder=2)

for col in df_away.columns:
    if "_x" in col or "_y" in col:
        df_away[col] = savgol_filter(df_away[col], window_length=25, polyorder=2)

In [32]:
df_away

,Period,Time [s],Away_1_x,Away_1_y,Away_2_x,Away_2_y,Away_3_x,Away_3_y,Away_4_x,Away_4_y,...,Away_8_x,Away_8_y,Away_9_x,Away_9_y,Away_10_x,Away_10_y,Away_11_x,Away_11_y,ball_x,ball_y
Frame,,,,,,,,,,,,,,,,,,,,,
0,1,0.00,-40.497385,-0.006452,-1.316036,-21.377255,-13.761025,-19.716595,-0.225905,0.243078,...,-12.687456,-4.450378,-12.091304,5.551665,-8.655242,-5.986888,-1.233523,21.292073,-53.0,34.0
1,1,0.04,-40.515915,-0.005300,-1.338186,-21.407557,-13.707752,-19.741814,-0.215049,0.172746,...,-12.673418,-4.486940,-12.086325,5.520446,-8.643539,-5.986120,-1.224848,21.305591,-53.0,34.0
2,1,0.08,-40.531689,-0.004098,-1.357143,-21.435754,-13.657000,-19.766093,-0.202435,0.107317,...,-12.660389,-4.522717,-12.082563,5.489741,-8.633024,-5.986671,-1.214929,21.317874,-53.0,34.0
3,1,0.12,-40.544707,-0.002845,-1.372909,-21.461848,-13.608767,-19.789431,-0.188064,0.046792,...,-12.648369,-4.557709,-12.080016,5.459549,-8.623697,-5.988541,-1.203766,21.328922,-53.0,34.0
4,1,0.16,-40.554968,-0.001543,-1.385484,-21.485838,-13.563055,-19.811828,-0.171935,-0.008830,...,-12.637360,-4.591916,-12.078686,5.429872,-8.615559,-5.991730,-1.191358,21.338733,-53.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,1,23.68,-36.081725,-2.849322,18.768589,-25.438874,6.227085,-27.864949,23.498657,-4.315132,...,-4.480930,-17.676827,13.291194,13.864848,17.921509,-16.698868,17.321913,16.342115,-53.0,34.0
593,1,23.72,-36.066612,-2.867001,18.785317,-25.421668,6.169990,-27.860428,23.526234,-4.330166,...,-4.456786,-17.670835,13.276519,13.873601,17.947848,-16.716821,17.211536,16.382815,-53.0,34.0
594,1,23.76,-36.050522,-2.885649,18.801962,-25.404700,6.115227,-27.857354,23.555214,-4.345715,...,-4.432568,-17.664314,13.259747,13.882304,17.975058,-16.736778,17.101421,16.424013,-53.0,34.0


In [33]:
df_home.to_csv('./results/home.csv', index=False)

In [34]:
df_away.to_csv('./results/away.csv', index=False)

In [36]:
PLOTDIR = './results'
mviz.save_match_clip(df_home.iloc[0:0+500],df_away.iloc[0:0+500],PLOTDIR,fname='result_1',include_player_velocities=False)

RuntimeError: Requested MovieWriter (ffmpeg) not available